In [ ]:
!pip install ultralytics

## Importing all the necessary libraries

In [ ]:
import os
import cv2
import yaml
import shutil
import numpy as np
import matplotlib.pyplot as plt

import torch
print(torch.cuda.is_available())

from ultralytics import YOLO

# Data exploring and pre-processing

## Getting directory tree structure

In [ ]:
def list_directory_tree_with_os_walk(starting_directory):
    for root, directories, files in os.walk(starting_directory):
        print(f"Directory: {root}")
list_directory_tree_with_os_walk('/kaggle/input/large-license-plate-dataset')

As we can see, the structure of the dataset is not similar to the one YOLO model needs to have for training. We need to fix it.

In [ ]:
shutil.copytree('/kaggle/input/large-license-plate-dataset/images/train/', '/kaggle/working/dataset/train/images')
shutil.copytree('/kaggle/input/large-license-plate-dataset/labels/train/', '/kaggle/working/dataset/train/labels')
shutil.copytree('/kaggle/input/large-license-plate-dataset/images/val/', '/kaggle/working/dataset/validation/images')
shutil.copytree('/kaggle/input/large-license-plate-dataset/labels/val/', '/kaggle/working/dataset/validation/labels')
shutil.copytree('/kaggle/input/large-license-plate-dataset/images/test/', '/kaggle/working/dataset/test/images')
shutil.copytree('/kaggle/input/large-license-plate-dataset/labels/test/', '/kaggle/working/dataset/test/labels')

Lets check the tree structure now:

In [ ]:
list_directory_tree_with_os_walk('/kaggle/working/dataset')

In [ ]:
training_config = {"path": "/kaggle/working/dataset", "train":"train/images", "val": "validation/images", "test": "test/images", "names": {0: "license_plate"}}
with open('data.yaml', 'w') as yaml_file:
    yaml.dump(training_config, yaml_file, sort_keys=False)

## Data exploring
Let's plot some pictures from the training set with the ground-truth bounding boxes.

In [ ]:
IMG_DATA = "/kaggle/working/dataset/{}/images"
LABELA_DATA = "/kaggle/working/dataset/{}/labels"

def preprocess_bbox(bbox_data, img_height, img_width):
    
    bbox_data = bbox_data.strip('\n')
    # class, bbox center x, bbox center y, h, w
    _, x, y, w, h = map(float, bbox_data.split(" "))
    x1 = int((x - w / 2) * img_width)
    x2 = int((x + w / 2) * img_width)
    y1 = int((y - h / 2) * img_height)
    y2 = int((y + h / 2) * img_height)
    
    return [x1, y1, x2, y2]   


def plot_labeled_data(mode='train'):

    fig = plt.figure(figsize=(20, 20)) 
    rows = 4
    columns = 4
    
    imgs_list = os.listdir(IMG_DATA.format(mode))
    labels_list = os.listdir(LABELA_DATA.format(mode))
    
    for i, img_name in enumerate(imgs_list[:16]):
        
        img = cv2.imread(os.path.join(IMG_DATA.format(mode), img_name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_h, img_w, _ = img.shape
        
        fl = open(os.path.join(LABELA_DATA.format(mode), img_name[:-3] + 'txt'), 'r')
        data = fl.readlines()
        for d in data:
            bbox = preprocess_bbox(d, img_h, img_w)
            cv2.rectangle(img=img, pt1=(bbox[0], bbox[1]), pt2=(bbox[2], bbox[3]), color=(255, 0, 155), thickness=2)
        fl.close()
        fig.add_subplot(rows, columns, i+1) 
        plt.imshow(img)
    plt.show()
    
plot_labeled_data()

**As we can see, some augmentations have been already done.**

In [ ]:
plot_labeled_data(mode='test')

In [ ]:
model = YOLO("yolov8s.pt")

## Training procedure

In [ ]:
import torch
import gc

# Clear cache
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# Optional: reset peak memory stats
torch.cuda.reset_peak_memory_stats()

In [ ]:
# results = model.train(data="data.yaml", epochs=30, imgsz=640, patience=10)
results = model.train(data="data.yaml", epochs=30, imgsz=640, patience=10, batch=0.9)

## Model evaluation on the test dataset
We will use `conf=0.15` and `iou=0.3` for inference.

In [ ]:
metrics = model.val(data='data.yaml', split='test', conf=0.15, iou=0.3)

In [ ]:
metrics.box.map,

## Plotting predictions

In [ ]:
predicted = model.predict(source='/kaggle/working/dataset/test/images', conf=0.15, iou=0.3, classes=[0])

In [ ]:
def preprocess_bbox(bbox, img_height, img_width):
    
    
    return [int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])]   


def plot_predicted_data(predicted):

    fig = plt.figure(figsize=(20, 20)) 
    rows = 4
    columns = 4
    
    
    for i, pred in enumerate(predicted):
        img_path = pred.path
        bboxes = pred.boxes.xyxy
        
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_h, img_w, _ = img.shape
        
        for bbox in bboxes:
            bbox = preprocess_bbox(bbox.tolist(), img_h, img_w)
            cv2.rectangle(img=img, pt1=(bbox[0], bbox[1]), pt2=(bbox[2], bbox[3]), color=(255, 0, 155), thickness=2)
        fig.add_subplot(rows, columns, i+1) 
        plt.imshow(img)
    plt.show()  
        

In [ ]:
plot_predicted_data(predicted[:16])

# Conclusion
That was a trainig procedure for YOLOv8-based model.
Model's perfomance can be improved at least my changing the net's architecture, size of dataset, etc.